In [3]:
import numpy as np

In [7]:
new = {}
old = {}
with open('horse-to-zebra-dist/train/real_B.txt','r') as f:
    for line in f:
        line = line.split(',')
        new[line[0]] = line[1:]
with open('horse-to-zebra-dist/train/real_B_1.txt','r') as f:
    for line in f:
        line = line.split(',')
        old[line[0]] = line[1:]

In [10]:
a = np.array([[1,2,2],[1,3,4]])
print (np.argmax(a[:,2]))

1


In [41]:
import scipy.io
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [194]:
import scipy.stats
import os

def chi_square(x,y):
#     x = x / np.sum(x)
#     y = y / np.sum(y)
    res = 0
    for i in range(x.shape[0]):
        a = (x[i] - y[i])**2
        if a == 0:
            continue
        a /= (x[i] + y[i])
        res += a
    return res / 2


def get_real(A=True):
    if A == True:
        real_A = os.listdir('/Users/Martin/Desktop/CS280/finalproj/photo-to-monet-dist/test/real_A')
    else:
        real_A = os.listdir('/Users/Martin/Desktop/CS280/finalproj/photo-to-monet-dist/test/real_B')
    real_A_first_hist = {}
    real_A_second_hist = {}

    count = 0
    A_average_first = np.zeros(32)
    A_average_second = np.zeros(32)
    for file in real_A:
        count += 1
        if A == True:
            data = scipy.io.loadmat('photo-to-monet-dist/test/real_A/'+file)
        else:
            data = scipy.io.loadmat('photo-to-monet-dist/test/real_B/'+file)
        first_mag = data['mag_first_layer']
        second_mag = data['mag_second_layer']
        first_ind = data['index_first_layer']
        second_ind = data['index_second_layerd']
        first = np.ndarray.flatten(first_ind.astype(int))
        first, _ = np.histogram(first, bins=32, range=(0,32))
        second = np.ndarray.flatten(second_ind.astype(int))
        second, _ = np.histogram(first, bins=32, range=(0,32))
        real_A_first_hist[file]=first
        real_A_second_hist[file]=second
        A_average_first += first
        A_average_second += second

    A_average_first /= count
    A_average_second /= count
    return real_A_first_hist, real_A_second_hist, A_average_first, A_average_second


def get_fake(A=True):
    if A == True:
        
        fake_A = os.listdir('/Users/Martin/Desktop/CS280/finalproj/photo-to-monet-dist/test/fake_A')
    else:
        fake_A = os.listdir('/Users/Martin/Desktop/CS280/finalproj/photo-to-monet-dist/test/fake_B')
    fake_A_first_hist = {}
    fake_A_second_hist = {}


    A_average_first = np.zeros(32)
    A_average_second = np.zeros(32)
    for file in fake_A:
        if A == True:
            data = scipy.io.loadmat('photo-to-monet-dist/test/fake_A/'+file)
        else:
            data = scipy.io.loadmat('photo-to-monet-dist/test/fake_B/'+file)
        first_mag = data['mag_first_layer']
        second_mag = data['mag_second_layer']
        first_ind = data['index_first_layer']
        second_ind = data['index_second_layerd']
        first = np.ndarray.flatten(first_ind.astype(int))
        first, _ = np.histogram(first, bins=32, range=(0,32))
        second = np.ndarray.flatten(second_ind.astype(int))
        second, _ = np.histogram(first, bins=32, range=(0,32))
        fake_A_first_hist[file]=first
        fake_A_second_hist[file]=second
    return fake_A_first_hist, fake_A_second_hist

def calculate_mean_A(real_A ,average_A):
    result = 0
    for key in real_A:

#         result += scipy.stats.entropy(real_A[key], average_A)
  
        result += chi_square(real_A[key], average_A)

    result /= len(real_A)
    return result

def calculate_var_A(fake_A, average_A):
    KL_div = np.zeros(len(fake_A))
    i = 0
    for key in fake_A:

#         KL_div[i] = scipy.stats.entropy(fake_A[key],average_A)

        KL_div[i] = chi_square(fake_A[key], average_A)
        i += 1
    return np.var(KL_div)


def calculate_score(real_A, fake_A, average_A):
    keys = fake_A.keys()
    result = np.zeros(len(fake_A))
    result_m1 = np.zeros(len(fake_A))
    i = 0
    for key in keys:

#         div = scipy.stats.entropy(fake_A[key], average_A)
        div = chi_square(fake_A[key], average_A)
        mean = calculate_mean_A(real_A, average_A)
        var = calculate_var_A(fake_A, average_A)
        
        m1_score = (div - mean) / var

        score = div
        result_m1[i] = m1_score
        result[i] = score
        i += 1
#     print(result)
    return result, result_m1, keys

real_A_first_hist, real_A_second_hist, A_average_first, A_average_second = get_real(False)   
fake_A_first_hist, fake_A_second_hist = get_fake(False)
first_res_fake_A, first_res_fake_A_m1, key_1_fake_A = calculate_score(real_A_first_hist, fake_A_first_hist, A_average_first)
second_res_fake_A, second_res_fake_A_m1, key_2_fake_A  = calculate_score(real_A_second_hist, fake_A_second_hist, A_average_second)
first_res_real_A, first_res_real_A_m1, key_1_real_A  = calculate_score(real_A_first_hist, real_A_first_hist, A_average_first)
second_res_real_A, second_res_real_A_m1, key_2_real_A  = calculate_score(real_A_second_hist, real_A_second_hist, A_average_second)

results = {}
results['1_layer_fake_B_keys'] = key_1_fake_A
results['2_layer_fake_B_keys'] = key_2_fake_A
results['1_layer_fake_B_m1'] = first_res_fake_A_m1
results['1_layer_fake_B_chi2'] = first_res_fake_A
results['2_layer_fake_B_m1'] = second_res_fake_A_m1
results['2_layer_fake_B_chi2'] = second_res_fake_A

results['1_layer_real_B_keys'] = key_1_real_A
results['2_layer_real_B_keys'] = key_2_real_A
results['1_layer_real_B_m1'] = first_res_real_A_m1
results['1_layer_real_B_chi2'] = first_res_real_A
results['2_layer_real_B_m1'] = second_res_real_A_m1
results['2_layer_reak_B_chi2'] = second_res_real_A

scipy.io.savemat('B.mat',results)




# np.save('1st_layer_B.npy',first_res)
# np.save('2nd_layer_B.npy',second_res)

# plt.hist(first_res)
# plt.xlabel('score')
# plt.ylabel('counts')
# plt.show()
# plt.hist(second_res)
# plt.xlabel('score')
# plt.ylabel('counts')
# plt.show()

        
    
